In [1]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from PIL import Image
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import GlobalAveragePooling2D,Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input, concatenate
from sklearn.model_selection import train_test_split
from keras.applications import ResNet50
from keras.layers import Embedding, Reshape
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import cv2
# from keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import OneHotEncoder
from joblib import Parallel, delayed
from tensorflow.keras.layers import Concatenate
from sklearn.model_selection import train_test_split

In [2]:
test_data = pd.read_csv('test.csv')
train_withbox_data = pd.read_csv('train_withbox_split_clean_encode.csv')

In [3]:
#train_data 열 타입 변경
# train_data['laterality']=train_data['laterality'].astype('string')
# train_data['view']=train_data['view'].astype('category')
# train_data['file_path']=train_data['file_path'].astype('string')
# train_data['cancer'] = train_data['cancer'].astype('int')
# train_data = train_data.dropna(subset=['age'])
# train_data = train_data.dropna(subset=['density'])
# train_data = train_data.dropna(subset=['BIRADS'])
# file_path 열 생성
train_withbox_data['image_path'] = train_withbox_data.apply(lambda row: f"test/{row.patient_id}_{row.image_id}.png", axis=1)

train_withbox_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54659 entries, 0 to 54658
Data columns (total 22 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Unnamed: 0               54659 non-null  int64 
 1   Unnamed: 0.1             54659 non-null  int64 
 2   site_id                  54659 non-null  int64 
 3   patient_id               54659 non-null  int64 
 4   image_id                 54659 non-null  int64 
 5   laterality               54659 non-null  object
 6   view                     54659 non-null  int64 
 7   age                      54659 non-null  int64 
 8   cancer                   54659 non-null  int64 
 9   biopsy                   54659 non-null  int64 
 10  invasive                 54659 non-null  int64 
 11  BIRADS                   54659 non-null  int64 
 12  implant                  54659 non-null  int64 
 13  density                  54659 non-null  int64 
 14  machine_id               54659 non-nul

In [4]:
train_withbox_data.head()

,Unnamed: 0,Unnamed: 0.1,site_id,patient_id,image_id,laterality,view,age,cancer,biopsy,...,implant,density,machine_id,difficult_negative_case,x,y,w,h,split,image_path
0,0,0,2,5,640805896,L,5,68,0,0,...,0,255,21,1,0,357,941,1982,1,test/5_640805896.png
1,1,1,2,5,940388076,R,1,68,0,0,...,0,255,21,0,1187,50,887,2149,1,test/5_940388076.png
2,2,2,2,5,1351088028,L,1,68,0,0,...,0,255,21,1,4,334,814,2094,1,test/5_1351088028.png
3,3,3,2,5,1633417959,R,5,68,0,0,...,0,255,21,0,1097,211,985,2157,1,test/5_1633417959.png
4,4,4,1,25,1789648218,L,1,62,0,0,...,0,1,49,0,13,563,1655,3080,0,test/25_1789648218.png


In [5]:
# merged_data = pd.merge(train_data, train_split, on=['patient_id', 'image_id'], how='inner', suffixes=('_train', '_split'))
# merged_data.to_csv('merged_data.csv', index=False)


In [ ]:
#이미지 전처리 수행 유방부분 사진만 편집후 test에 저장
import cv2
import glob
import numpy as np
from tqdm import tqdm
import os
import re
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
from concurrent.futures import ProcessPoolExecutor
from joblib import Parallel, delayed

def fit_image(fname):
    X = cv2.imread(fname)
    # Some images have narrow exterior "frames" that complicate selection of the main data. Cutting off the frame
    X = X[5:-5, 5:-5]
    # regions of non-empty pixels
    output= cv2.connectedComponentsWithStats((X > 20).astype(np.uint8)[:, :, 0], 8, cv2.CV_32S)
    stats = output[2]
    idx = stats[1:, 4].argmax() + 1
    x1, y1, w, h = stats[idx][:4]
    x2 = x1 + w
    y2 = y1 + h
    X_fit = X[y1: y2, x1: x2]
    patient_id, im_id = re.findall('(\d+)_(\d+).png', os.path.basename(fname))[0]
    os.makedirs('test', exist_ok=True)
    cv2.imwrite(f'test/{patient_id}_{im_id}.png', X_fit[:, :, 0])
    return True

def fit_all_images(all_images):
    Parallel(n_jobs=4)(delayed(fit_image)(image_path) for image_path in all_images)

if __name__ == '__main__':
    
    fit_all_images(train_withbox_data['image_path'])

    np.random.seed(123)
    for fname in np.random.choice(glob.glob('test/*'), size=5):
        plt.figure(figsize=(20, 10))
        patient_id, im_id = re.findall('(\d+)_(\d+).png', os.path.basename(fname))[0]
        plt.suptitle(f'[{fname}]')
        im1 = Image.open(fname).convert('F')
        plt.subplot(121).imshow(im1, cmap='gray')
        plt.subplot(121).set_title(f'Output image {im1.size}')
        im2 = Image.open(f'{train_withbox_data[train_withbox_data["image_path"].str.contains(f"{patient_id}_{im_id}")]["image_path"].values[0]}').convert('F')
        plt.subplot(122).imshow(im2, cmap='gray')
        plt.subplot(122).set_title(f'Source image {im2.size}')
        plt.show()


In [ ]:
# 이미지 크기 설정
img_width, img_height = 300, 400

# 이미지 로드 및 리사이징 함수
def load_and_resize_image(img_path):
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = cv2.resize(img, (img_width, img_height))
    img = img.reshape((img_width, img_height, 1))
    return img

# 데이터를 4개의 그룹으로 분할 (L-1, L-5, R-1, R-5)
groups = {'L1': [], 'L5': [], 'R1': [], 'R5': []}
labels = {'L1': [], 'L5': [], 'R1': [], 'R5': []}

for _, row in train_withbox_data.iterrows():
    image_path = row['image_path']
    image = load_and_resize_image(image_path)
    group_key = f"{row['laterality']}{row['view']}"
    groups[group_key].append(image)
    labels[group_key].append(row['cancer'])


In [ ]:
# 각 그룹에 대해 모델을 학습하고 평가
for group_key in groups.keys():
    images = np.array(groups[group_key])
    labels_group = np.array(labels[group_key])

    # 데이터를 훈련 및 테스트 데이터로 분할
    train_images, test_images, train_labels, test_labels = train_test_split(images, labels_group, test_size=0.2, random_state=42)

    # 기본 모델 정의 (ResNet50)
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 1))

    # Flatten Layer 추가
    flat_layer = Flatten()(base_model.output)

    # 분류를 위한 Dense Layer 추가
    output = Dense(1, activation='sigmoid')(flat_layer)

    # 모델 생성
    model = Model(inputs=base_model.input, outputs=output)

    # 모델 컴파일
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    # 모델 학습
    model.fit(train_images, train_labels, epochs=5, batch_size=32)

    # 모델 평가
    loss, accuracy = model.evaluate(test_images, test_labels)
    print(f'{group_key} - Test Loss: {loss}, Test Accuracy: {accuracy}')
